# Laboratorio 2

- **Fecha Inicio**: Miércoles 25 de abril del 2018
- **Fecha Informe**: Domingo 06 de mayo del 2018

## Instalación

Para facilitar el proceso de instalación de esta actividad, trabajaremos con una máquina virtual que tendrá _casi_ todos los programas instalados.

Como motor de máquinas virtuales usaremos [Virtual Box](https://www.virtualbox.org/wiki/Downloads). Desde ese link tendrán que descargar la versión que mejor se ajuste a su sistema operativo. Luego desde los servidores de la universidad deben descargar la [máquina virtual](http://niebla.ing.puc.cl/diplomadobigdata/vbox-pyreclab-lda.ova) ya configurada.

Finalmente debemos importar la máquina descargada dentro de Virtual Box, para ello deben seguir los siguientes pasos: Abrir virtual box > Archivo > Abrir servicio virtualizado, o bien `Crtl+I`.

**Observación:** la contraseña del usuario configurado es _ubuntu_.

Una vez abierto la máquina virtual, abrir la consola o terminal y escribir los siguiente comandos:

```bash
$ sudo apt-get update
$ sudo apt-get install python3-dev
$ sudo apt-get install git
$ sudo pip3 install jupyter
$ sudo pip3 install matplotlib
$ sudo pip3 install pandas
$ sudo pip3 install scipy
$ sudo pip3 install cython
$ sudo pip3 install implicit
$ sudo pip3 install sklearn
```

Descargar este proyecto ya sea con `git` o mediante el botón de descargar y ejecutar `notebook`.

```bash
$ git clone https://github.com/stgolarrain/recsys-labs.git
$ cd recsys-labs/assignment-2
$ jupyter notebook
```

## Instrucciones

En este laboratorio vamos a utilizar la librería [implicit](https://github.com/benfred/implicit), la cual implementa dos modelos de *implicit feedback* con [Bayesian Probabilistic Ranking](https://arxiv.org/pdf/1205.2618.pdf) y [Alternating Least Square (ALS)](https://web.stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf).

Por otro lado, aprenderás a usar la librería [pandas](https://pandas.pydata.org/) para la manipulación de datos a través de *data frames*. En la tercera parte, usaremos la librería [scikit-learn](http://scikit-learn.org/) la cual es considerada un estándar en temas de `machine learning` y modelos predictivos. Recomendamos al alumno revisar la documentación de ambas librerías.
 
También contarán con un pequeño set de datos ubicado en la carpeta `/dataset`, el cual incluye dos archivos: `movies.dat` y `ratings.dat`. Como es de esperarse el archivo `movies.dat` cuenta con metadatos de películas, mientras que `ratings.dat` contiene los *ratings* que diferentes usuarios le asignaron a diferentes películas. El formato de cada archivo es como sigue:

- `movies.dat`: `movie_id; title; genres`
- `ratings.dat`: `user_id; movie_id; rating; timestamp`

Los datos de `genres` corresponden a las categorías de la película separados por el caracter |.

En este laboratorio tendrán que implementar las siguientes tareas:

1. Descripción preliminar del set de datos
2. Entrenar y evaluar cualitativamente los modelos de factorización utilizando ambos métodos
3. Visualizar los vectores latentes mediante la técnica PCA

## 1. Descripción de los Datos

En esta sección utilizaremos la librería [Pandas](https://pandas.pydata.org/) para el manejo de *data frames*.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Load movies dataset
movies_path = './dataset/movies.dat'
headers = ['movie_id', 'title', 'genres']
movies = pd.read_csv(movies_path,
                     names=headers,
                     delimiter=';')

movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
print('# Movies:', movies['movie_id'].nunique())

**Pregunta** ¿Cuántas películas tiene el set de datos?

**Respuesta**

In [5]:
# Load rating dataset
ratings_path = './dataset/ratings.dat'
headers = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_path,
                     names=headers,
                     delimiter=';',
                     usecols=['user_id', 'movie_id', 'rating'])

ratings[:5]

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [8]:
# Load rating dataset
ratings_path = './dataset/movies.dat'
headers = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_path,
                     names=headers,
                     delimiter=';',
                     usecols=['user_id', 'movie_id', 'rating'])

ratings[:5]
# print('# Set de Datos:', len(ratings))

         user_id  movie_id  rating
0              1      1193       5
1              1       661       3
2              1       914       3
3              1      3408       4
4              1      2355       5
5              1      1197       3
6              1      1287       5
7              1      2804       5
8              1       594       4
9              1       919       4
10             1       595       5
11             1       938       4
12             1      2398       4
13             1      2918       4
14             1      1035       5
15             1      2791       4
16             1      2687       3
17             1      2018       4
18             1      3105       5
19             1      2797       4
20             1      2321       3
21             1       720       3
22             1      1270       5
23             1       527       5
24             1      2340       3
25             1        48       5
26             1      1097       4
27             1    

**Pregunta** ¿Cuántos usuarios tiene el set de datos?

** Respuesta**

In [ ]:
# TODO: Code your answer here

**Pregunta** ¿Cuál es el porcentaje de densidad que tiene la matriz usuario-items?

** Respuesta**

In [ ]:
# Top 5 most popular movies
top_5_count = ratings.groupby(['movie_id'], as_index=False)[['rating']].count().head(5)
top_5_count = top_5_count.merge(movies, on='movie_id', how='inner')
top_5_count

In [ ]:
# Top 5 most like movies (average rating)
top_5_mean_rating = ratings.groupby(['movie_id'], as_index=False)[['rating']].mean().head(5)
top_5_mean_rating = top_5_mean_rating.merge(movies, on='movie_id', how='inner')
top_5_mean_rating

**Pregunta** ¿Cuales son las películas menos populares en términos de cantidad?

*Hint* Te recomendamos utilizar la función [`tail`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.tail.html) de Pandas

**Respuesta**

In [ ]:
# TODO: Code your answer here

**Pregunta** ¿Cuales son las películas menos populares en términos de rating promedio?

**Respuesta**

In [ ]:
# TODO: Code your answer here

**Pregunta** Describa las listas de películas menos populares ¿Se parecen las listas de películas menos populares? ¿Por qué?

**Respuesta**

In [ ]:
# Rating Histogram
ratings['rating'].hist(bins=5)
plt.title('Rating Histogram')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()

**Pregunta**: ¿El set de datos tiene problemas de `cold-start` o ítems con pocos ratings? Justifique generando un histograma con los ratings por usuario y por ítems.

*Hint*: recomendamos utilizar el parámetro bins en el histograma para visualizar la respuesta.

**Respuesta**

In [ ]:
# TODO: Code your answer

## 2. Factorización Matricial

In [ ]:
import implicit
import numpy as np
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking

In [ ]:
# Parse data into a sparse matrix
data = ratings

# Generate map for user
unique_users = data['user_id'].unique()
user_to_int = {u: i for i, u in enumerate(unique_users)}
int_to_user = dict(enumerate(unique_users))

# Generate map for movies
unique_movies = data['movie_id'].unique()
movie_to_int = {u: i for i, u in enumerate(unique_movies)}
int_to_movie = dict(enumerate(unique_movies))

# Map user and movies
user_data = data['user_id'].map(lambda u: user_to_int[u]).tolist()
movie_data = data['movie_id'].map(lambda m: movie_to_int[m]).tolist()

data = coo_matrix((data['rating'].astype(np.float32),
                  (movie_data, user_data)))
items = data.T.tocsr()

En la próxima celda generaremos nuestro modelo ALS y lo entrenaremos con los datos de forma implícita.

In [ ]:
# Train an ALS model
als_model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=50)
als_model.fit(data)

En la próxima celda generaremos nuestro modelo BPR y lo entrenaremos con los datos de forma implícita.

In [ ]:
# Train a BPR model
bpr_model = BayesianPersonalizedRanking(factors=50, regularization=0.01, iterations=50)
bpr_model.fit(data)

A continuación se implementaron dos funciones que permite generar recomendaciones para un usuario dado un modelo, y otra que permite ver las 10 películas con mejor rating para un usuario. Deberá generar recomendaciones para un usuario utilizando los modelos BPR y ALS; visualizar las películas con mejor rating y responder las siguientes preguntas

In [ ]:
# Don't change this cell
def recommend(user_id, model, items, movies, movie_map):
    recommendations = model.recommend(user_id, items)
    df_data = {
        'movie_id': [],
        'scores': []
    }

    for movie_id, score in recommendations:
        df_data['movie_id'].append(movie_map[movie_id])
        df_data['scores'].append(score)
        
    df = pd.DataFrame(data=df_data)
    df = df.merge(movies, on=['movie_id'], how='inner')
        
    return df
    
def show_user(user_id, ratings, movies, movie_map, user_map):
    user_id = user_map[user_id]
    df = ratings[ ratings['user_id'] == user_id].sort_values(['rating'], ascending=False)
    df = df.merge(movies, on=['movie_id'], how='inner')
    return df[:10]

# Usage example
# 
# user_id = 0 
# recommend(user_id, als_model, items, movies, int_to_movie)
# show_user(user_id, ratings, movies, int_to_movie, int_to_user)

In [ ]:
# TODO: Code your answer

**Pregunta**: Según tu punto de vista, ¿qué modelo obtuvo la mejor recomendación?

**Respuesta**

**Pregunta**: Según el contenido visto en clases, ¿por qué un modelo genera mejor recomendaciones que otro? Justifique su respuesta.

**Respuesta**:

## 3. Visualización de vectores latentes

En esta sección tomaremos los vectores latentes del modelo BPR y aplicaremos una reducción de dimensionalidad a 2D mediante PCA. Luego de ejecutar las próximas celdas, deberá responder las preguntas.

In [ ]:
# Don't change this cell
from sklearn.decomposition import PCA

# Train a PCA model
pca = PCA(n_components=2)
data = pca.fit_transform(bpr_model.item_factors)

plt.scatter(data[:, 0], data[:, 1])
plt.title('Movie Embeddings Plot')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
N = 20

# Don't change the under this line
labels = []
categories = []

for i in range(N):
    movie_id = int_to_movie[i]
    movie_name = movies[ movies['movie_id'] == movie_id ]['title'].tolist()[0]
    genres = movies[ movies['movie_id'] == movie_id ]['genres'].tolist()[0]
    
    labels.append(movie_name)
    categories.append(genres)
    

plot_data = data[:N]
plt.figure(figsize=(20,20))
plt.scatter(plot_data[:, 0], plot_data[:, 1])
plt.title('Movie Embeddings Plot')
plt.xlabel('x')
plt.ylabel('y')

# Plot movie names
for i in range(N):
    plt.annotate(labels[i], (plot_data[i, 0], data[i, 1] + .01))
    plt.annotate(categories[i], (plot_data[i, 0], data[i, 1] - .01))

plt.show()

**Pregunta**: Explique en sus palabras qué información capturan los vectores latentes de ítem.

**Respuesta**:

**Pregunta**: En base al gráfico anterior, asigne un nombre o explicación a los ejes `x` e `y`.

**Respuesta**:

**Pregunta**: Dado lo aprendido en clases sobre PCA, explique cómo se obtuvieron los ejes `x` e `y`.

**Respuesta**:

## Entregable

Una vez completado el laboratorio y respondido las preguntas deberán exportar este archivo en formato `html` y subir a la plataforma _Moodle_.

Para exportar este archivo deben ir a `File > Donwload as > HTML (.html)`

Si tienen algún problema o duda enviar mail a **dparra [at] ing [dot] puc [dot] cl** o **slarrain [at] uc [dot] cl** anteponiendo [Diplomada Bog Data] en el asunto.